# DATA 620: Final 
     
## Building a pipeline for exploring and visualizing authorship and publishing practices in astrophysics
#### John DeBlase, Daina Bouquin
   
### Background on data sources
The [SAO/NASA Astrophysics Data System (ADS)](https://ui.adsabs.harvard.edu/), is an online database of over eight
million astronomy and physics papers from both peer reviewed and non-peer reviewed sources. The ADS is a highly used resource in the Astronomy and Physics communities and has many levels of indexing. The ADS API makes it possible to query this valuable resource to better understand authorship and publishing behavior in these fields among many other applications. In the near future, the ADS plans to begin incorporating Unified Astronomy Thesaurus (UAT) keywords into their indexing schema. The ADS is managed by the Smithsonian Astrophysical Observatory at the Harvard–Smithsonian Center for Astrophysics.

### Problem formulation
Are there notable differences in centrality measures among networks of authors and publications in various astrophysical domains?   

### Methods
Below, we demonstrate how we used the ADS API to obtain publication data on highly cited articles in three different astrophysical domains: Cosmology, High Energy Astrophysics, and Exoplanet Astronomy. These topics were chosen because they are top-level categories in the [Unified Astronomy Thesaurus (UAT)](http://astrothesaurus.org/), which will be soon integrated into the indexing proceedures of major publishers like the [American Astronomical Society](https://aas.org/publications/publishing) and subsequently used as a searchable field within the ADS. You can visualize the [full UAT taxonomy here](http://uat.altbibl.io/sort/). In order to limit necessary processing time and develop this workflow as a proof-of-concept, the comparisons between neworks were limited to these three domains. 
   
After the ADS queries were executed, the resulting data was restructured into a SQL database (using SQLite), which was subsequently used to construct graph objects that are displayed using [sigma.js](http://sigmajs.org/). The graph is created using NetworkX. Degree centrality, betweeness, and PageRank are cacluated to show how publication tendencies differ between journals on a given topic and which authors create the most bridges between publications. These steps aggregated into a Flask app which can be run locally as specified below.

### Processing
The complete code showing the data extraction, restructuring, analysis, and eventual visualization is available [here](https://github.com/bsnacks000/NasaADS_graphviz).
   
In order to make our Flask application scale quickly it was decided early on that each graph would be stored in a SQLite database as a seperate set of node and edge tables for each query based on subject. The backend processing API takes place off server and consists of two classes, ProcessADS and ProcessGraph, which are responsible for formatting the tables as csv files.

First the ProcessADS class is instantiated. It hits the ADS API, cleans and parses the raw query into a set of node and edge tables. These are what will become the 'main' graph for each subject. 

This object is then passed into a new instance of ProcessGraph. This class performs all the neccesary networkx calculations to create bipartite connected component subgraphs and finally export the csv files. In just several lines of code, 8 correctly formatted node and edge tables can be generated for each subject. The process is also designed to be transparent, so that node and edge data can be inspected by the user throughout the data warehousing and cleaning process.

The second phase of processing involves creating a set of sqlalchemy object relational models for each subject table. A utility function creates reads the orm classes in order to create the schema for the database. 

Finally odo, an optimized data pipeline toolkit, is used to load all the csv data into the SQLite database that sits on the server. 

The Flask server is a simple single endpoint RESTful API. Once the user submits the form data specifying their subject and graph on the web page, Flask loads the table into an object instantiated with the PrepareSigma class. This class automatically adds several extra fields such as 'x', 'y' and 'color' via pandas and numpy that are specifically used by sigma.js on the frontend. This final object is sent to the browser via json and rendered by sigma.


### Running the Graph Vizualization Tool
The visualization tool can be built and run locally via the following instructions:   

1. clone [this repo](https://github.com/bsnacks000/NasaADS_graphviz)
2. create a fresh [virtual env](http://docs.python-guide.org/en/latest/dev/virtualenvs/) in that directory
3. activate the venv (tested on Python 2.7.9 + 2.7.11)
4. pip install dependencies
5. Run ```python app.py``` in the top level of the App directory to boot the server to localhost:5000

To just run the app with the ready made database, you can simply pip install flask, numpy and pandas with an installation of Python 2.7.9 +
```
pip install flask numpy pandas
```
If you want to re-build the database or add more subjects using our processing classes in the Data Cleaning top level folder you will need to add several more packages to manually run scripts
```
pip install ads sqlalchemy odo jupyter
```
Alternatively you can install everything using requirements.txt
```
pip install -r requirements.txt
```

Instructions for building the database can be found each of the ipython notebooks in the [Data Cleaning folder](https://github.com/bsnacks000/NasaADS_graphviz/tree/master/DataCleaning). Be forewarned that this process is not automatic and will require the user to correctly create additional .py files in the "orm" folder.


### Demonstrating the tool's proof-of-concept through simple comparisons:
    
#### 1. Visualizing the nodes with the highest degree and betweeness centrality measure across domains

### Cosmology
<img src="images/main_cosmology.png">
### High Energy Astrophysics
<img src="images/main_highenergy.png">
### Exoplanet Astronomy
<img src="images/main_exoplanet.png">

As the graphs show, regardless of domain, for most highly cited papers in a given field, the publications with the highest degree and betweeness centrality measures is the Astrophysical Journal. This is interesting for a number of reasons, the first of which is the fact that the highest impact journal ([Impact Factors](https://en.wikipedia.org/wiki/Impact_factor) are used to determine the relative impact of a given publication) is not the Astrophysical Journal, but Nature:

**Nature** 2015 Impact Factor: 38.138   
**ApJ** 2015 Impact Factor: 5.909   

The significance of this observation is many-fold. For as long as impact factors have existed, there have been critics of their actual ability to represent the impact of a paper on a domain, and yet impact factor is still a highly referenced metric that drives publication practices among researchers. Researchers try very hard to publish in "high impact journals" to gain recognition and justify their work for promotions and funding applications. From our visualization though we demonstrate how flawed this notion is; in a given domain, the "impact" of a journal is not necessarily reflected in Impact Factor.

#### 2. Betweeness of author nodes
   
It is also useful to examine betweeness centrality of authors within the publication network. Betweeness is a value equal to the number of shortest paths from all vertices to all others that pass through that node. This is to day that a node with high betweenness centrality has a large influence on the transfer of items through the network, under the assumption that item transfer follows the shortest paths. This is useful for us to determine the influence of a given author on their domain's publication behavior.

### Cosmology
<img src="images/main_authors_cos.png">
### High Energy Astrophysics
<img src="images/main_authors_hea.png">
### Exoplanet Astronomy
<img src="images/main_authors_exo.png">

As seen in the screenshots above, some authors are more collaborative and have a higher degree of betweeness among their peers. By identifying these highly influential authors and displaying their collaborations independent of impact factor it is easier for the researcher to make the case that their work is highly influential. It would also be interesting to identify other metrics about these authors to show how other attributes like age, gender, and affiliation may be predictive of high degrees of centrality in a given domain.

### Future Analysis and Limitations

There are a number of avenues that could be explored using the protocol and platform defined above. For instance, it would be useful to examine and explore differences between journals based on the funding model of the journal publisher as a way of contextualizing conversations about open access publishing, as well as alternate funding models, and impact measures in scholarly publishing. Additional background information would need to be gathered and timelines would need to be taken into account; this is to say that the funding model and publishing practices of a given journal may change over time, and the criteria for our initial query is based on citation counts, but is not limited in any way by time. Additionally, because the data exists in a static state, the comparisons described herein represent only a snapshot of the data existing in the ADS. Similarly, the results are limited in their representation of the domain being named for each graph. Until the UAT is fully implemented by all major publishers the search strategy employed in the inital stages of the processing pipeline are full text and keyword searches. Those keyword searches are not necessarily defined as part of a controlled vocabulary across domains.